In [1]:
#####可変基底非負値テンソル分解#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
import gc
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [76]:
####データの生成####
##データと日付idの設定
#データの設定
k = 10
m = 3
hh = 3000
location = 1500
week = 7
month = 31
year = 12
max_week = 5
month_days = np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
year_days = np.sum(month_days)

#日付IDを設定
day_id = np.tile(np.arange(week), int((year_days*m)/week))[:m*year_days]
week_id = np.tile(np.array(list(itertools.chain(*[np.repeat(np.arange(max_week), week)[:month_days[j]] for j in range(year)]))), m)
month_id = np.tile(np.repeat(np.arange(year), month_days), m)
date_n = np.array([week, max_week, year])

In [83]:
##パラメータの生成
#事前分布の設定
gamma = np.array([3.0, 1.0, 0.25, 0.5])
alpha = np.array([0.3])
beta = np.array([0.75])

#モデルパラメータの生成
omega = np.random.dirichlet(gamma, hh)
theta_u = np.random.gamma(alpha, 1/beta, hh*k).reshape(hh, k)
theta_v = np.random.gamma(alpha, 1/beta, location*k).reshape(location, k)
theta_d = [j for j in range(m+1)]
for j in range(m):
    theta_d[j] = np.random.gamma(alpha, 1/beta, date_n[j]*k).reshape(date_n[j], k)
theta_d[m] = np.repeat(1.0, k)

In [81]:
theta_d[m]

3